In [10]:
import numpy as np
import pandas as pd
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split,KFold
from sklearn.ensemble import RandomForestRegressor
import time
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso




In [2]:
dataset=pd.read_csv("train.csv")


In [3]:
o3=dataset["OZONE"]
no2=dataset["NO2"]

In [4]:
time1=dataset["Time"]
unix_timestamps = [dt.datetime.strptime(ts, '%Y-%m-%d %H:%M:%S').timestamp() for ts in time1]
some_features=dataset.iloc[:,5:]
all_features=dataset.iloc[:,3:] 
all_features = pd.concat([all_features, pd.Series(unix_timestamps)],axis=1)
all_features = all_features.rename(columns={0: 'Time'})
# print(some_features)
# print(all_features)


In [6]:
test_data=pd.read_csv("dummy_test.csv")
time1=test_data["Time"]
unix_timestamps = [dt.datetime.strptime(ts, '%Y-%m-%d %H:%M:%S').timestamp() for ts in time1]
test_all=test_data.iloc[:,3:]
test_all = pd.concat([test_all, pd.Series(unix_timestamps)],axis=1)
test_all = test_all.rename(columns={0: 'Time'})
test_features=test_data.iloc[:,5:]
test_o3=test_data["OZONE"]
test_no2=test_data["NO2"]

In [8]:
min_o3=np.min(o3)
max_o3=np.max(o3)
min_no2=np.min(no2)
max_no2=np.min(no2)


## Linear Regression

In [32]:
model_o3=LinearRegression()
model_no2=LinearRegression()

model_o3.fit(all_features,o3)
model_no2.fit(all_features,no2)

pred_o3=model_o3.predict(all_features)
pred_no2=model_no2.predict(all_features)

mae_no2 = mean_absolute_error(no2, pred_no2)
mae_o3 = mean_absolute_error(o3, pred_o3)

print(f"MAE of NO2 {mae_no2} MAE of O3 {mae_o3} ")


MAE of NO2 6.401693285490042 MAE of O3 5.186773650546027 


In [69]:
model_o3=LinearRegression()
model_no2=LinearRegression()

scaler = StandardScaler()
X_train = scaler.fit_transform(some_features)
X_test = scaler.transform(some_features)

lasso_no2 = Lasso(alpha=0.1)
lasso_no2.fit(some_features, no2)
y_pred_no2 = lasso_no2.predict(some_features)
mae_no2 = mean_absolute_error(no2, y_pred_no2)
print('MAE of NO2:', mae_no2)

# Train a Lasso regression model for O3
lasso_o3 = Lasso(alpha=0.1)
lasso_o3.fit(some_features, o3)
y_pred_o3 = lasso_o3.predict(some_features)
mae_o3 = mean_absolute_error(o3, y_pred_o3)
print('MAE of O3:', mae_o3)

C:\Users\Yash\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 186570.70991113572, tolerance: 235.49076648388356
  model = cd_fast.enet_coordinate_descent(


MAE of NO2: 6.534249928499446
MAE of O3: 5.627557675371524


C:\Users\Yash\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3306.6620221980847, tolerance: 707.672918246965
  model = cd_fast.enet_coordinate_descent(


## For O3 you have to use all features and for no2 you have to use only 4 features
## Decision Tree

In [27]:
X_train, X_test, y_train, y_test = train_test_split(all_features, o3, test_size=0.2)
X_train1, X_test1, y_train1, y_test1 = train_test_split(all_features, no2, test_size=0.2)

model_o3 = DecisionTreeRegressor(max_depth=11)
model_no2 = DecisionTreeRegressor(max_depth=11)

model_o3.fit(X_train, y_train)
model_no2.fit(X_train1,y_train1)

pred_o3 = model_o3.predict(X_test)
pred_no2 = model_no2.predict(X_test1)

mae_no2 = mean_absolute_error(y_test1, pred_no2)
mae_o3 = mean_absolute_error(y_test, pred_o3)
r2_no2 = r2_score(y_test1, pred_no2)
r2_o3 = r2_score(y_test, pred_o3)

print(f"MAE of NO2 {mae_no2} MAE of O3 {mae_o3} ")
print(f"R2 score of NO2 {r2_no2} and R2 score of O3 {r2_o3} ")
with open('model_o3.pkl', 'wb') as f:
    pickle.dump(model_o3, f)
with open('model_no2.pkl', 'wb') as f:
    pickle.dump(model_no2, f)
    
model_size=os.path.getsize('model_o3.pkl')
model_size+=os.path.getsize('model_no2.pkl')
model_size

MAE of NO2 2.8255330905132188 MAE of O3 4.038019898644441 
R2 score of NO2 0.7638767517112978 and R2 score of O3 0.895287868529178 


343250

## KNN

In [ ]:
k = 5  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(all_features, o3, test_size=0.2)
X_train1, X_test1, y_train1, y_test1 = train_test_split(all_features, no2, test_size=0.2)

model_o3 = KNeighborsRegressor(n_neighbors=11)
model_no2 = KNeighborsRegressor(n_neighbors=11)

for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    X_fold_train, X_fold_val = X_train[train_index], X_train[val_index]
    y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
    model_o3.fit(X_fold_train, y_fold_train)
    
    
    # Here you can train and evaluate your model using X_fold_train, y_fold_train, X_fold_val, and y_fold_val
    # For simplicity, let's print the fold number and the sizes of the training and validation sets
    print(f"Fold {fold+1} - Train: {len(X_fold_train)}, Validation: {len(X_fold_val)}")

# After training and evaluating on the validation sets, you can use X_test and y_test for final evaluation
print(f"Test set - Size: {len(X_test)}")
MAE of NO2 1.9481532727272728 MAE of O3 3.2231490454545457 


In [58]:
model_o3 = KNeighborsRegressor(n_neighbors=5)
model_no2 = KNeighborsRegressor(n_neighbors=5)

X_train, X_test, y_train, y_test = train_test_split(all_features, o3, test_size=0.25)
X_train1, X_test1, y_train1, y_test1 = train_test_split(all_features, no2, test_size=0.25)

model_o3.fit(X_train, y_train)
model_no2.fit(X_train1,y_train1)

pred_o3 = model_o3.predict(X_test)
pred_no2 = model_no2.predict(X_test1)

mae_no2 = mean_absolute_error(y_test1, pred_no2)
mae_o3 = mean_absolute_error(y_test, pred_o3)
r2_no2 = r2_score(y_test1, pred_no2)
r2_o3 = r2_score(y_test, pred_o3)

print(f"MAE of NO2 {mae_no2} MAE of O3 {mae_o3} ")
print(f"R2 score of NO2 {r2_no2} and R2 score of O3 {r2_o3}" )

MAE of NO2 1.6493794800000003 MAE of O3 3.2978188800000003 
R2 score of NO2 0.9275290445918974 and R2 score of O3 0.9296052621801095


In [17]:
X_train, X_test, y_train, y_test = train_test_split(all_features, o3, test_size=0.2)
X_train1, X_test1, y_train1, y_test1 = train_test_split(all_features, no2, test_size=0.2)

model_o3 = KNeighborsRegressor(n_neighbors=12)
model_no2 = KNeighborsRegressor(n_neighbors=12)
tic=time.time()
model_o3.fit(X_train, y_train)
model_no2.fit(X_train1,y_train1)

pred_o3 = model_o3.predict(X_test)
pred_no2 = model_no2.predict(X_test1)

mae_no2 = mean_absolute_error(y_test1, pred_no2)
mae_o3 = mean_absolute_error(y_test, pred_o3)
r2_no2 = r2_score(y_test1, pred_no2)
r2_o3 = r2_score(y_test, pred_o3)

with open('model_o3.pkl', 'wb') as f:
    pickle.dump(model_o3, f)
with open('model_no2.pkl', 'wb') as f:
    pickle.dump(model_no2, f)
print(time.time()-tic)    
print(f"MAE of NO2 {mae_no2} MAE of O3 {mae_o3} ")
print(f"R2 score of NO2 {r2_no2} and R2 score of O3 {r2_o3}" )

0.14892148971557617
MAE of NO2 1.9456624791666666 MAE of O3 3.151254729166667 
R2 score of NO2 0.9120815834326994 and R2 score of O3 0.9326390990070587


In [23]:
import pickle
X_train, X_test, y_train, y_test = train_test_split(all_features, o3, test_size=0.2)
X_train, X_test, y_train1, y_test1 = train_test_split(all_features, no2, test_size=0.2)
all_features=pd.DataFrame(all_features)
# train model
tic=time.time()
model = KNeighborsRegressor(n_neighbors=1)
model.fit(X_train, np.column_stack((y_train, y_train1)))

# predict on test set
pred = model.predict(X_test)
pred_o3 = pred[:, 0]
pred_no2 = pred[:, 1]

# calculate metrics
mae_o3 = mean_absolute_error(y_test, pred_o3)
mae_no2 = mean_absolute_error(y_test1, pred_no2)
print(time.time()-tic)

# save model using pickle
with open("final", "wb") as f:
    pickle.dump(model, f)

# print metrics
print(f"MAE of NO2: {mae_no2}")
print(f"MAE of O3: {mae_o3}")

0.04811239242553711
MAE of NO2: 1.46755375
MAE of O3: 19.463340499999997


## Random Forest

In [18]:
import pickle,os
X_train, X_test, y_train, y_test = train_test_split(all_features, o3, test_size=0.2)
X_train1, X_test1, y_train1, y_test1 = train_test_split(all_features, no2, test_size=0.2)

rf_o3 = RandomForestRegressor( max_depth=20, n_estimators=200)
rf_no2 = RandomForestRegressor( max_depth=20, n_estimators=200)
tic=time.time()
rf_o3.fit(X_train,y_train)
o3_mae_train = mean_absolute_error(y_test, rf_o3.predict(X_test))




rf_no2.fit(X_train1,y_train1)
no2_mae_train = mean_absolute_error(y_test1, rf_no2.predict(X_test1))
toc=time.time()

print('MAE on training set for NO2:', no2_mae_train)
print('MAE on training set for O3:', o3_mae_train)

print(toc-tic)
with open('model_o3.pkl', 'wb') as f:
    pickle.dump(rf_o3, f)
with open('model_no2.pkl', 'wb') as f:
    pickle.dump(rf_no2, f)
model_size=os.path.getsize('model_o3.pkl')
model_size+=os.path.getsize('model_no2.pkl')

model_size

MAE on training set for NO2: 1.6769377198313096
MAE on training set for O3: 3.1551465440604636
19.609964847564697


411820030

In [20]:
import datetime as dt
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import time
import numpy as np
import pandas as pd
import pickle

# Load the dataset
dataset = pd.read_csv("train.csv")

# Separate the target variables
o3 = dataset["OZONE"]
no2 = dataset["NO2"]

# Convert time to Unix timestamps
time1 = dataset["Time"]
unix_timestamps = [dt.datetime.strptime(ts, '%Y-%m-%d %H:%M:%S').timestamp() for ts in time1]

# Concatenate the features and the timestamps
all_features = dataset.iloc[:, 3:]
all_features.insert(0, 'Time', unix_timestamps)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_features, np.column_stack((o3, no2)), test_size=0.2)



In [22]:
# Train the model
model = KNeighborsRegressor(n_neighbors=2)
tic = time.time()
model.fit(X_train, y_train)
pred = model.predict(X_test)
pred_o3 = pred[:, 0]
pred_no2 = pred[:, 1]

# Calculate metrics
mae_o3 = mean_absolute_error(y_test[:, 0], pred_o3)
mae_no2 = mean_absolute_error(y_test[:, 1], pred_no2)
print(time.time() - tic)

# Save the model using pickle
with open("final", "wb") as f:
    pickle.dump(model, f)

# Print metrics
print(f"MAE of NO2: {mae_no2}")
print(f"MAE of O3: {mae_o3}")
print(os.path.getsize("final"))

0.05886483192443848
MAE of NO2: 1.3949895
MAE of O3: 3.2431185000000005
2324448
